In [ ]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

#This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

import hail as hl
hl.init(spark_conf={'spark.driver.memory': '10G', 'spark.executor.memory': '10G'}, default_reference='GRCh38') 

In [131]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis

from hail.experimental.vcf_combiner.vcf_combiner import combine_gvcfs as cmb

output_notebook() 

Loading BokehJS ...

Loading BokehJS ...

In [118]:
import os
files = os.listdir('/data/data/gvcf/')

In [104]:
gvcfs = []
for f in files:
    if ((f.find("tbi")) == -1) & (f.find('txt') == -1):
        gvcfs.append('/data/data/gvcf/'+f)
        

In [105]:
gvcfs.sort()

In [106]:
len(gvcfs)

186

In [107]:
header_try=open('/data/data/gvcf/header.txt').read()

In [127]:
samples = []

for f in files:
    if ((f.find("tbi")) == -1) & (f.find('txt') == -1):
        samples.append((f.split('.'))[0])

samples.sort()

In [ ]:
hl.experimental.run_combiner(gvcfs, out_file='/data/data/joint/gts-sparse.mt',
                             tmp_path='/tmp', reference_genome='GRCh38', use_genome_default_intervals = True,
                             header = '/data/data/gvcf/header.txt',
                             sample_names = samples)

2021-11-09 19:02:30 Hail: INFO: Using 2586 intervals with default whole-genome size 1200000 as partitioning for GVCF import
2021-11-09 19:02:30 Hail: INFO: GVCF combiner plan:
    Branch factor: 100
    Batch size: 100
    Combining 186 input files in 2 phases with 2 total jobs.
        Phase 1: 1 job corresponding to 2 intermediate output files.
        Phase 2: 1 job corresponding to 1 final output file.

2021-11-09 19:02:30 Hail: INFO: Starting phase 1/2, merging 186 input GVCFs in 1 job.
2021-11-09 19:02:30 Hail: INFO: Starting phase 1/2, job 1/1 to create 2 merged files, corresponding to ~50.0% of total I/O.


In [132]:
sparse = hl.read_matrix_table('/data/data/joint/gts-sparse.mt')

In [133]:
sparse.count()

(2923228397, 186)

In [134]:
dense = hl.experimental.densify(sparse)

dense.write('/data/data/joint/gts-dense.mt')

In [136]:
gts = hl.read_matrix_table('/data/data/joint/gts-dense.mt')

In [137]:
gts.count()

(2923228397, 186)

In [140]:
gts_gt = gts.annotate_entries(GT=hl.experimental.lgt_to_gt(gts.LGT, gts.LA))

In [ ]:
gts_gt.write('/data/data/joint/gts-geno.mt')

In [142]:
gts_gt = hl.read_matrix_table('/data/data/joint/gts-geno.mt')

In [143]:
gts_gt.count()

(2923228397, 186)

In [146]:
pheno = hl.import_table('/data/data/external-data/familiespattern.csv')

2021-11-15 17:15:15 Hail: WARN: Name collision: field 'sample' already in object dict. 
  This field must be referenced with __getitem__ syntax: obj['sample']
2021-11-15 17:15:15 Hail: INFO: Reading table without type imputation
  Loading field 'sample' as type str (not specified)
  Loading field 'family' as type str (not specified)
  Loading field 'mutation_option_1' as type str (not specified)
  Loading field 'mutation_option_2' as type str (not specified)


In [153]:
to_keep = pheno['sample'].collect()

In [154]:
gts_gt = gts_gt.filter_cols(hl.literal(to_keep).contains(gts_gt.s))

In [157]:
gts_gt = gts_gt.filter_rows(hl.agg.any(gts_gt.GT.is_non_ref()))

In [ ]:
gts_gt.write('/data/data/joint/fams-unfiltered.mt')

In [159]:
gts_gts = hl.read_matrix_table('/data/data/joint/fams-unfiltered.mt')

In [160]:
gts_gts.count()

(35472229, 124)